In [ ]:
# import webdriver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import os
import glob
from astropy.io import fits
import numpy as np

## load the coordinates
with fits.open(f'/Users/eusracenorth/Downloads/xgap_m500_forecast.fits') as f:
    srcids = f[1].data['ID']
    ras = f[1].data['RA_C']  # Right Ascension in decimal degrees
    decs = f[1].data['DEC_C']  # Declination in decimal degrees


for i in range(len(srcids)):
    ra = np.round(ras[i], decimals=4)
    dec = np.round(decs[i], decimals=4)

    #### Download on rosat website #### 
    url = f'https://heasarc.gsfc.nasa.gov/cgi-bin/Tools/xraybg/xraybg.pl?Entry={ra}%2C{dec}&NR=GRB%2FSIMBAD%2BSesame%2FNED&CoordSys=J2000&radius=1.00&region=cone&inner_radius=n%2Fa&spectrum=create&scaling=hist'
    # create webdriver object
    # Set up ChromeOptions for headless mode
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Run Chrome in headless mode
    driver = webdriver.Chrome(options=chrome_options)

    driver.get(url)

    # Find specific links based on their text content
    # links_to_download = driver.find_elements(By.XPATH, '//a[text()="ROSAT PSPCc response matrix"]')

    # Download files
    links = driver.find_elements(By.XPATH, '//a[text()="ROSAT PSPCc response matrix"]')
    for link in links:
        link.click()
        time.sleep(5)

    links = driver.find_elements(By.XPATH, '//a[text()="FITS counts spectrum"]')
    for link in links:
        link.click()
        time.sleep(5)

    # Close the browser window
    driver.quit()
    
    savepath = f'/Users/eusracenorth/Downloads/ROSAT_specs/ID{srcids[i]}'
    os.makedirs(savepath, exist_ok=True)

    os.system(f'mv *.pi {savepath}/rosat.pi')
    os.system(f'mv *.rsp {savepath}/rosat.rsp')


In [ ]:
from astropy import coordinates as coord
import astropy.units as u
import numpy as np

def deci2gala(ra_decimal, dec_decimal):
    # Create an Astropy SkyCoord object
    icrs = coord.SkyCoord(ra=ra_decimal*u.degree, dec=dec_decimal*u.degree, frame='icrs')

    # Convert to Galactic coordinates
    galactic_coords = icrs.transform_to(coord.Galactic)

    if galactic_coords.b.degree>0:
        # Print the Galactic coordinates
        return f'{np.round(galactic_coords.l.degree, decimals=4)}', f'+{np.round(galactic_coords.b.degree, decimals=4)}'
    else:
        return f'{np.round(galactic_coords.l.degree, decimals=4)}', f'{np.round(galactic_coords.b.degree, decimals=4)}'



In [ ]:
# Input celestial coordinates in decimal degrees
ra_decimal = 169.088  # Right Ascension in decimal degrees
dec_decimal = 29.22171  # Declination in decimal degrees

deci2gala(ra_decimal, dec_decimal)